In [3]:
import pandas as pd
import os

# 기본 설정
DATA_DIR = "../data/processed"
OUTPUT_PATH = os.path.join(DATA_DIR, "grid_features_extended.csv")

# ① 기준 데이터
df = pd.read_csv(os.path.join(DATA_DIR, "demand_supply_analysis.csv"))

# ② 전기차 등록 수 (시군구/읍면동 → grid_id 매핑 불가 시 생략)
# (※ 현재는 직접 매핑된 데이터 없으므로 보류 또는 추후 spatial join 필요)

# ③ 공영주차장 수 (grid_id 기준)
parking_df = pd.read_csv(os.path.join(DATA_DIR, "charging_stations_seoul_gridded.csv"))
parking_count = (
    parking_df[parking_df['시설구분(소)'] == "공영주차장"]
    .groupby("grid_id")
    .size()
    .reset_index(name="public_parking_count")
)

df = df.merge(parking_count, on="grid_id", how="left")
df["public_parking_count"] = df["public_parking_count"].fillna(0).astype(int)

# ① 데이터 로드
hourly_df = pd.read_csv(os.path.join(DATA_DIR, "charging_hourly_processed.csv"))

# ② '정보없음' 필터링
hourly_df = hourly_df[
    (hourly_df['충전시작시간'] != '정보없음') &
    (hourly_df['충전종료시간'] != '정보없음')
]

# ③ 문자열을 datetime으로 변환
hourly_df['충전시작시간'] = pd.to_datetime(hourly_df['충전시작시간'], errors='coerce')
hourly_df['충전종료시간'] = pd.to_datetime(hourly_df['충전종료시간'], errors='coerce')

# ④ NaT가 된 행 제거
hourly_df = hourly_df.dropna(subset=['충전시작시간', '충전종료시간'])

# ⑤ 평균 시간당 충전량 계산
hourly_df['duration_hours'] = (hourly_df['충전종료시간'] - hourly_df['충전시작시간']).dt.total_seconds() / 3600
hourly_df = hourly_df[hourly_df['duration_hours'] > 0]  # 음수 제거
hourly_df['charging_rate'] = hourly_df['충전량(kW)'] / hourly_df['duration_hours']
avg_charging_rate = hourly_df['charging_rate'].mean()


df["avg_hourly_load"] = avg_charging_rate

# 저장
df.to_csv(OUTPUT_PATH, index=False)
print(f"✅ 저장 완료: {OUTPUT_PATH}")

✅ 저장 완료: ../data/processed/grid_features_extended.csv
